In [21]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# load train and test dataset
def load_dataset():
    # load the dataset
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    # one hot encode target values
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return X_train, y_train, X_test, y_test

In [22]:
def normalize(X_train, X_test):
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    return X_train, X_test

In [23]:
import os
import time

LOG_ROOT = 'train_logs'

def generate_log_dir(model_name):
    run_id = time.strftime(f"{model_name}_%Y_%m_%d-%H_%M_%S")
    return os.path.join(LOG_ROOT, run_id)

In [29]:
# load dataset
X_train, y_train, X_test, y_test = load_dataset()

# normalize the features
X_train, X_test = normalize(X_train, X_test)

In [24]:
import tensorflow as tf

def run_test_harness(create_model_fn, model_name):
    # create the model
    model = create_model_fn()

    # fit the model
    model_filepath = f"models/{model_name}.h5"

    callbacks = []
    callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=generate_log_dir(model_name)))
    callbacks.append(tf.keras.callbacks.ModelCheckpoint(model_filepath, save_best_only=True))
    model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks)

    # evaluate the model
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {(acc * 100.0):.3f}")

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD

# this is the baseline model with 1 VGG block
def create_vgg_1_block_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))

    # compile the model
    optimizer = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [26]:
run_test_harness(create_vgg_1_block_model, 'cifar10_vgg_1_block')

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 7s 136us/sample - loss: 1.7363 - accuracy: 0.3786 - val_loss: 1.5155 - val_accuracy: 0.4561
Epoch 2/100
50000/50000 [==============================] - 6s 124us/sample - loss: 1.4359 - accuracy: 0.4914 - val_loss: 1.3600 - val_accuracy: 0.5147
Epoch 3/100
50000/50000 [==============================] - 6s 123us/sample - loss: 1.2771 - accuracy: 0.5465 - val_loss: 1.2360 - val_accuracy: 0.5613
Epoch 4/100
50000/50000 [==============================] - 6s 124us/sample - loss: 1.1623 - accuracy: 0.5893 - val_loss: 1.1829 - val_accuracy: 0.5854
Epoch 5/100
50000/50000 [==============================] - 6s 121us/sample - loss: 1.0844 - accuracy: 0.6201 - val_loss: 1.1015 - val_accuracy: 0.6102
Epoch 6/100
50000/50000 [==============================] - 6s 120us/sample - loss: 1.0124 - accuracy: 0.6456 - val_loss: 1.1333 - val_accuracy: 0.5981
Epoch 7/100
50000/50000 [===================

In [27]:
# this is a model with 2 VGG blocks
def create_vgg_2_blocks_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))

    # compile the model
    optimizer = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [28]:
run_test_harness(create_vgg_2_blocks_model, 'cifar10_vgg_2_blocks')

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 9s 178us/sample - loss: 1.7070 - accuracy: 0.3856 - val_loss: 1.4890 - val_accuracy: 0.4651
Epoch 2/100
50000/50000 [==============================] - 8s 164us/sample - loss: 1.3627 - accuracy: 0.5142 - val_loss: 1.2998 - val_accuracy: 0.5397
Epoch 3/100
50000/50000 [==============================] - 8s 167us/sample - loss: 1.1916 - accuracy: 0.5815 - val_loss: 1.1345 - val_accuracy: 0.6002
Epoch 4/100
50000/50000 [==============================] - 9s 172us/sample - loss: 1.0638 - accuracy: 0.6278 - val_loss: 1.0852 - val_accuracy: 0.6138
Epoch 5/100
50000/50000 [==============================] - 9s 175us/sample - loss: 0.9683 - accuracy: 0.6632 - val_loss: 1.0327 - val_accuracy: 0.6411
Epoch 6/100
50000/50000 [==============================] - 9s 181us/sample - loss: 0.8998 - accuracy: 0.6867 - val_loss: 0.9654 - val_accuracy: 0.6647
Epoch 7/100
50000/50000 [===================